Dhananjay Tiwari,
Ph.D. Mechanical Engineering,
Mechanical Engineering Laboratory, 1204
University of Illinois at Urbana Champaign

This file has the code for implementation of FLPO optimization problem using using scipy optimizers

In [53]:
# include libraries and dependencies
import numpy as np # eh! computational python programming and not include this? Can't happen!
from scipy import optimize # optimization module in scipy
from scipy.optimize import minimize # minimize function using various algorithms

In [76]:
# create FLPO class and define its associated functions
class FLPO():

    # define data members of the class
    N : int # number of vehicles
    M : int # number of nodes
    K : int # number of stages
    uav_id : list # vehicle indices
    node_id : list # facility indices
    stop_id : list # destination id
    node_param : dict # node parameters

    stages : list # FLPO stages, K elements
                  #   Nx1  (M+1)x1  (M+1)x1   (M+1)x1    (M+1)x1       1x1
                  #   stg0   stg1    stg2     stg3 . . . . stgK-2     stgK-1
                  #    0      0        0        0  . . . .  0               
                  #    1      1        1        1  . . . .  1
                  #    2      2        2        2  . . . .  2          dest
                  #    .      .        .        .  . . . .  .
                  #    .      .        .        .  . . . .  .
                  #    .      M-1      M-1      M-1  . . .  M-1
                  #    .      dest    dest     dest  . . .  dest
                  #    N-1    

    def __init__(self, N, M):
        # this function intializes the FLPO structure with assigned values
        
        # UAVs, nodes and stopping state indices
        self.N = N # initialize the number of vehicles
        self.M = M # initialize the number of nodes
        self.K = self.M + 2 # 1 initial stage + #intermediate stages equal to the number of facilities + 1 final stage
        self.uav_id = list(range(N)) # 0 1 2 . . . N-1
        self.node_id = list(range(N, N+M)) # N N+1 . . . N+M-1
        self.stop_id = list(range(N+M, N+M+1)) # N+M
        
        # initialize parameters associated to all the facilities
        self.node_param = dict()
        # self.fac_param['locations'] = np.zeros([M, 2])
        self.node_param['schedules'] = np.zeros([M+1, N]) # for each node, there are n-time parameters for each vehicle

        # define the structure of the stages and the elements in them
        self.stages = [0 for element in range(self.K)] # initialize list stages. Each 0 to be replaced with a column vector
        for stg in range(self.K):
            if stg == 0:
                # initial stage consisting of nodes
                self.stages[stg] = self.uav_id # Nx1 array
            elif stg == self.K-1:
                # final stage consists of only stopping state
                self.stages[stg] = self.stop_id # 1x1 array
            else:
                # other stages consist of facilities and destination
                self.stages[stg] = self.node_id + self.stop_id # (M+1) x 1 array
        
        # initialize association probabilities
        self.asn_pb = [[0 for element1 in range(self.K-1)] for element2 in range(N)]
        for n in range(N): # iteration over each node in the initial stage
            for stg in range(self.K-1): # iteration goes until the penultimate stage
                next_stage = self.stages[stg+1] # get the next stage nodes
                # initialize probabilities as ones (normalized in the next step)
                if stg == 0:
                    self.asn_pb[n][stg] = np.ones([1, len(next_stage)])
                else:
                    self.asn_pb[n][stg] = np.ones([len(self.stages[stg]), len(self.stages[stg+1])])
                # hence normalize the probability associations
                for i in range(np.shape(self.asn_pb[n][stg])[0]):
                    self.asn_pb[n][stg][i, :] = self.asn_pb[n][stg][i, :]/len(self.asn_pb[n][stg][i, :]) # normalize probabilities

    # define stage transportation cost for each node        
    def cost_stage_transit(self, n, stg, i, j):
        # this function gives the value of stage transition cost for node n
        # from node i in stage stg to node j in stage stg+1
        # for now the cost is defined based on the facility schedules

        t = self.node_param['schedules'] # a 2D matrix with each row representing a facility, and every column representing a node
        # get the indices of i and j and vehicle n
        id_i = self.stages[stg].index(i)
        id_j = self.stages[stg+1].index(j)
        id_n = self.uav_id.index(n)

        if i == j:
            # self-hopping has very high penalty
            cost = 100
        elif i == self.stop_id and j != self.stop_id:
            # vehicle cannot go to a node other than stopping state from the stopping state
            cost = 100
        else:
            # normal case
            cost = t[id_j, id_n] - t[id_i, id_n] # time taken to move from node i to node j by vehicle n

        return cost
    
    # define the distortion function of a node
    def distortion(self, n): # depends on probability associations and parameters of facilities
        # this function computes the distortion function of a node n
        if n not in self.uav_id:
            raise('invalid node id')
        else:
            # initialize the distortion data structure (same as stages data structure)
            D = [np.zeros(len(self.stages[stg])) for stg in range(self.K)]

            # iterate over stages : final stage --> initial stage
            stg_ids = list(range(self.K))
            stg_ids_rev = stg_ids[::-1]
            for stg in stg_ids_rev:
                print('stage -- ' , stg)
                # iterate over each element of the chosen stage
                for i in self.stages[stg]:
                    id_i = self.stages[stg].index(i)
                    print('i -- ' , i, id_i)
                    D[stg][id_i] = 0
                    if stg == self.K-1:
                        D[stg][id_i] = 0
                    else:
                        print('next stage nodes -- ')
                        for j in self.stages[stg+1]:
                            id_j = self.stages[stg+1].index(j)
                            D[stg][id_i] = D[stg][id_i] + self.asn_pb[n][stg][id_i, id_j]* \
                                (self.cost_stage_transit(n, stg, i, j))
        return D
    

    def entropy(self, n): # depends on probability associations
        return 0
    
    def free_energy():
        return 0
    
    def flat_asn_pb(self, asn_pb):
        # this function unwraps the association probabilities into a column vector

        vec = []
        # unwrap for a vehicle, and then put them all together
        for uav_pb in asn_pb:
            for stg_pb in uav_pb:
                vec = vec + list(stg_pb.flatten())

        return vec
    
    def inv_flat_asn_pb(self, vec_asn_pb):
        # this function wraps the association probabilities into their original defined structure
        
        return asn_pb
    
    def objective(self, x):
        # this function computes the objective of the FLPO problem from distortion of each vehicle and entropy 
        # of probability associations
        asn_pb = x
        # assign the association probabilities and node parameters 
        self.asn_pb = asn_pb
        self.node_param = params

        # initialize the total cost
        cost = 0

        # initialize the weights given to each vehicle
        uav_weights = np.ones(len(self.uav_id))/len(self.uav_id)

        # compute distortion cost by performing weighted sum over each vehicle
        for n in self.uav_id:
            id_n = self.uav_id.index(n)
            distortion_cost = distortion_cost + uav_weights[id_n] * self.distortion(n)
        cost = distortion_cost
        return cost



Minimize the distortion of FLPO with respect to probability associations and node schedule parameters

In [78]:
# create an instance of FLPO class
N = 2 # number of vehicles
M = 3 # number of nodes
flpo = FLPO(N, M) 
print(' -- Stages --\n' , flpo.stages)
print(' \n-- Association probabilities --\n' , flpo.asn_pb)
vec_asn_pb = flpo.flat_asn_pb(flpo.asn_pb)
print(len(vec_asn_pb))
# res = minimize(flpo.objective, init_condition) 

 -- Stages --
 [[0, 1], [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5], [5]]
 
-- Association probabilities --
 [[array([[0.25, 0.25, 0.25, 0.25]]), array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[1.],
       [1.],
       [1.],
       [1.]])], [array([[0.25, 0.25, 0.25, 0.25]]), array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), array([[1.],
       [1.],
       [1.],
       [1.]])]]
80
